# Server VITS on Trition Docker
- 선수 노트북: 아래를 먼저 실행해야 합니다
    - 0.0-create-tts-vits-model.ipynb

## 1.  Setup environment
사용하는 패키지는 import 시점에 다시 재로딩 합니다.

In [1]:
%load_ext autoreload
%autoreload 2

import sys, os
sys.path.append(os.path.abspath("./vits"))

for i in sys.path:
    print(i)

/home/ec2-user/SageMaker/lab/00-trition-tts-vits/02-tts-vits-docker-trition
/home/ec2-user/anaconda3/envs/conda-vits-py310/lib/python310.zip
/home/ec2-user/anaconda3/envs/conda-vits-py310/lib/python3.10
/home/ec2-user/anaconda3/envs/conda-vits-py310/lib/python3.10/lib-dynload

/home/ec2-user/anaconda3/envs/conda-vits-py310/lib/python3.10/site-packages
/home/ec2-user/SageMaker/lab/00-trition-tts-vits/02-tts-vits-docker-trition/vits


# 2. Trition 서빙 준비

## config.pbtxt 생성
* TRITON 서버의 모델 설정 파일을 작성합니다. 

In [2]:
import os

os.makedirs("workspace", exist_ok=True)

In [3]:
%%writefile workspace/config.pbtxt
name: "tts-vits" #모델 이름
platform: "pytorch_libtorch" #PyTorch 모델 사용
max_batch_size: 1 #최대 배치 크기
input [
  { # 입력 텍스트
    name: "x"
    data_type: TYPE_INT64 # 정수형
    dims: [ -1 ] # 가변 길이
  },
  {  # 텍스트길이
    name: "x_length"
    data_type: TYPE_INT64
    dims: [ 1 ]
    reshape: { shape: [] }
  },
  { # 음성 변화도
    name: "noise_scale"
    data_type: TYPE_FP32
    dims: [ 1 ]
    reshape: { shape: [] }
  },
  {
    name: "length_scale"
    data_type: TYPE_FP32
    dims: [ 1 ]
    reshape: { shape: [] }
  },
  {
    name: "noise_scale_w"
    data_type: TYPE_FP32
    dims: [ 1 ]
    reshape: { shape: [] }
  }
]
output [ #출력 설정
  {
    name: "OUTPUT_0" #출력명
    data_type: TYPE_FP32
    dims: [ 1, -1 ]
  }
]
instance_group [
  {
    count: 1
    kind: KIND_GPU #GPU 사용
  }
]
dynamic_batching {
  preferred_batch_size: [ 1 ] #선호하는 배치 크기
  max_queue_delay_microseconds: 100 #최대 대기 시간
}
default_model_filename: "model.pt" # 모델 파일 이름

Overwriting workspace/config.pbtxt


# 3. 아티펙트 패키징

## 모델 리파지토리 폴더 구조
```
model_serving_folder
    - model_name
        - version_number
            - model file
        - config file

# Example
hello-serve-pt
    - hello
        - 1
            - model.pt
        - config.pbtxt

```


##  아티펙트 폴더 생성 
* TRITON 서버를 위한 모델 서빙 폴더 구조를 생성하고 필요한 파일들을 복사하는 과정
    * 이 구조는 TRITON 서버의 요구사항을 준수하는 것으로 1/각 모델은 자신의 폴더를 갖고 2/모델 버전은 숫자폴더에 저장하고 3/설정파일은 모델 폴더에 위치하게 합니다.

In [4]:
import os
from local_utils.triton_util import make_folder_structure, copy_artifact, remove_folder

# 1. 변수 설정
# triton-hello-serve-pt 폴더 생성
workspace_folder ="workspace" #작업폴더
model_serving_folder = 'triton-serve-jit'#TRITON 서버 폴더
model_name = "tts-vits" #모델 이름
model_file_name = "trace_vits.pt" #모델파일이름

# 2. 파일 경로 설정
model_path = os.path.join(workspace_folder, model_file_name) #모델 파일 경로

model_config_path = os.path.join(workspace_folder, 'config.pbtxt')#설정 파일 경로

# 3. 폴더 구조 생성
make_folder_structure(model_serving_folder, model_name=model_name) 

 #copy_artifact(model_serving_folder, model_name, trace_model_name, model_config)
copy_artifact(model_serving_folder=model_serving_folder, 
            model_name=model_name, 
            model_artifact=model_path, 
            config=model_config_path)

triton-serve-jit:
tts-vits

triton-serve-jit/tts-vits:
1
config.pbtxt

triton-serve-jit/tts-vits/1:
model.pt


### 폴더 삭제
- 필요시 주석 제거하고 사용하세요.

In [5]:
# model_serving_folder = 'triton-serve-jit'
# remove_folder(model_serving_folder)

# 4. 로컬 도커에서 실행 테스트

## 4.0. 도커에서의 실행 테스트는 아래와 같은 순서로 진행 함.


#### (1) 터미널 실행

아래에서 둘 중의 한개의 도커를 실행 하시면 됩니다.

#### (2) SageMaker Triton 도커 컨테이너 실행 -- triton 24.05 version :
* ECR에서 TRITON 서버 도커 이미지를 가져오는 과정 1/ECR 로그인(get-loginp-password) 2/도커이미지 가져오기(pull) 
* 3/TRITON서버를 자세한 로깅 옵션과 함께 실행하기 (docker run \)
```
aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 763104351884.dkr.ecr.us-east-1.amazonaws.com

docker pull 763104351884.dkr.ecr.us-east-1.amazonaws.com/sagemaker-tritonserver:24.05-py3

# Move to current folder (e.g.: /home/ec2-user/SageMaker/lab/00-trition-tts-vits/ )

docker run --gpus=1 --rm -p8000:8000 -p8001:8001 -p8002:8002 -v `pwd`/triton-serve-jit:/models 763104351884.dkr.ecr.us-east-1.amazonaws.com/sagemaker-tritonserver:24.05-py3 tritonserver --model-repository=/models --log-verbose=3 --log-info=1 --log-warning=1 --log-error=1
```

#### Option:  NVidia Triton 도커 컨테이너 실행
- 위의 터미널에 아래와 같이 명령어를 하나씩 실행 하세요.
```
docker run --gpus=1 --rm -p8000:8000 -p8001:8001 -p8002:8002 -v `pwd`/triton-serve-jit:/models nvcr.io/nvidia/tritonserver:22.08-py3 tritonserver --model-repository=/models --log-verbose=3 --log-info=1 --log-warning=1 --log-error=1
```


#### (3) 도커 관련 유용한 명령어
```
docker rm -f $(docker ps -qa)

# 도커 실행하여 들어가 보기
docker run -it --entrypoint /bin/bash nvcr.io/nvidia/tritonserver:22.08-py3

# 실행중인 도커 컨테이너 들어가기
docker exec -it <container_name_or_id> /bin/bash

# 실시간 로그 확인
docker logs -f <container_id>

# 특정 에러 검색
docker logs <container_id> | grep ERROR

# 특정 모델 관련 로그 검색
docker logs <container_id> | grep tts-vits

```

## 4.1. !!! #### 터미널에 "Triton 도커 컨테이너 실행" 을 해주세요. ### !!!

# 5. Run Inference on Triton Client

## Triton Client 초기화
* TRITON 서버에 연결하기 위한 클라이언트 설정 : http, gRPC 클라이언트 동시 설정
* HTTP클라이언트 vs gRPC 클라이언트
    * HTTP: 간단한 설정, 디버깅 용이, 웹브라우저 테스트 가능
    * gRPC : 더 빠른 성능, 스트리밍 지원, 양방향 통신

In [6]:
from local_utils.triton_util import setup_triton_client
triton_client, grpcclient = setup_triton_client()

## Import Lib for converting text to token ID 
* VITS 모델을 사용하기 위한 기본 설정과 텍스트 전처리를 위한 함수 정의 과정

In [7]:
#필요한 라이브러리 임포트 # 주피터 노트북에서 그래프 표시
%matplotlib inline 
import matplotlib.pyplot as plt # 시각화 라이브러리
import IPython.display as ipd #오디오 재생용
import torch #PyTorch
import commons #VITS 공통함수
import utils #VITS 유틸리티 함수
from text import text_to_sequence #텍스트 전처리 함수

#텍스트 전처리 함수
def get_text(text, hps):
    #텍스트를 숫자 시퀀스로 바꿈
    text_norm = text_to_sequence(text, hps.data.text_cleaners)
    #blank 토큰 추가 (있는 경우)
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    #PyTorch 텐서로 변환    
    text_norm = torch.LongTensor(text_norm)
    return text_norm

#모델 설정 로드 : LJ Speech 데이테셋용 설정 파일 로드
hps = utils.get_hparams_from_file("vits/configs/ljs_base.json")

### Define Text to be spoken

In [8]:
# text = "Claude is AI for all of us. Whether you're brainstorming alone or building with a team of thousands, Claude is here to help"
text = "Today, we're announcing the Claude 3 model family, which sets new industry benchmarks across a wide range of cognitive tasks. The family includes three state-of-the-art models in ascending order of capability: Claude 3 Haiku, Claude 3 Sonnet, and Claude 3 Opus. Each successive model offers increasingly powerful performance, allowing users to select the optimal balance of intelligence, speed, and cost for their specific application."

## Create input variables

In [9]:
import numpy as np

def create_input_data_padding(text, hps, noise_scale, noise_scale_w, length_scale, max_sequence_length=2048):
    #1.텍스트 전처리
    stn_tst = get_text(text, hps) # "hello" --> [12,5,12,3,4,5]
    x_tst = stn_tst #변환된 시퀀스를 x_tst에 할당

    #2.텍스트 길이 정보 생성
    original_length = stn_tst.size(0)
    x_tst_lengths = torch.LongTensor([original_length]).cuda() # 시퀀스 길이를 텐서로 변환하고 GPU로 이동

    #3.NumPy 배열로 변환
    x_np = x_tst.detach().cpu().numpy() #GPU > CPU, 텐서 > NumPy
    print("x_np length", len(x_np))
    #4.패딩 처리
    if len(x_np) > max_sequence_length:
        print(f"Warning: Input length {len(x_np)} exceeds maximum length {max_sequence_length}. Truncating...")
        x_np = x_np[:max_sequence_length]
    elif len(x_np) < max_sequence_length:
        # 패딩 추가
        padding_length = max_sequence_length - len(x_np)
        print("padding_length: ", padding_length)
        x_np = np.pad(x_np, (0, padding_length), mode='constant', constant_values=0)

    print("x_np length after padding", len(x_np))        

    x_np = x_np.reshape(1,-1) #(N,) > (1,N)형태로 변환
    # x_length_np = x_tst_lengths.detach().cpu().numpy() #길이 정보는 원래 길이 유지
    # x_length_np = [max_sequence_length].numpy()
    x_length_np = np.array([max_sequence_length], dtype=np.int64) #음성 변화도

    #4.생성 파라미터 설정
    #각 파라미터를 float32 타입 NumPy 배열로 변환
    noise_scale_data = np.array([noise_scale], dtype=np.float32) #음성 변화도
    length_scale_data = np.array([noise_scale_w], dtype=np.float32) #발화 속도
    noise_scale_w_data = np.array([length_scale], dtype=np.float32) #음성 다양성

    # (1,N)크기의 텍스트 시퀀스, (1,) 크기의 길이정보, (1,)크기의 음성 변화도 (1,)크기의 발화속도 (1,)크기의 음성 다양성
    return (x_np, x_length_np, noise_scale_data, length_scale_data, noise_scale_w_data)   

    

   #함수 호출 예시:
input_vars = create_input_data_padding(text, hps, 
                              noise_scale=.667, 
                              noise_scale_w=0.8, 
                              length_scale=1,
                              max_sequence_length=2048
                              )
input_vars



x_np length 965
padding_length:  1083
x_np length after padding 2048


(array([[ 0, 62,  0, ...,  0,  0,  0]]),
 array([2048]),
 array([0.667], dtype=float32),
 array([0.8], dtype=float32),
 array([1.], dtype=float32))

In [10]:
x_np, x_length_np, noise_scale_data, length_scale_data, noise_scale_w_data = input_vars
# print variables
print("x_np shape:", x_np.shape)
print("x_np sample:", x_np[:5])  # Adjust the slice as needed

print("x_length_np shape:", x_length_np.shape)
print("x_length_np:", x_length_np)

print("noise_scale_data shape:", noise_scale_data.shape)
print("noise_scale_data:", noise_scale_data)

print("length_scale_data shape:", length_scale_data.shape)
print("length_scale_data:", length_scale_data)

print("noise_scale_w_data shape:", noise_scale_w_data.shape)
print("noise_scale_w_data:", noise_scale_w_data)

x_np shape: (1, 2048)
x_np sample: [[ 0 62  0 ...  0  0  0]]
x_length_np shape: (1,)
x_length_np: [2048]
noise_scale_data shape: (1,)
noise_scale_data: [0.667]
length_scale_data shape: (1,)
length_scale_data: [0.8]
noise_scale_w_data shape: (1,)
noise_scale_w_data: [1.]


In [11]:
# 전체 배열 보기
print("Full array:")
np.set_printoptions(threshold=np.inf)  # 모든 요소 출력
print(x_np)

Full array:
[[  0  62   0  83   0  46   0 156   0  47   0 102   0   3   0  16   0  65
    0 102   0 123   0  16   0  70   0  56   0 156   0  43   0 135   0  56
    0  61   0 102   0 112   0  16   0  81   0  83   0  16   0  53   0  54
    0 156   0  76   0 158   0  46   0  16   0 119   0 123   0 156   0  51
    0 158   0  16   0  55   0 156   0  69   0 158   0  46   0  83   0  54
    0  16   0  48   0 156   0  72   0  55   0 102   0  54   0  51   0   3
    0  16   0  65   0 157   0 102   0  62   0 131   0  16   0  61   0 156
    0  86   0  62   0  61   0  16   0  56   0 156   0  63   0 158   0  16
    0 156   0 102   0  56   0  46   0 138   0  61   0  62   0 123   0  51
    0  16   0  44   0 156   0  86   0  56   0  62   0 131   0  55   0  69
    0 158   0 123   0  53   0  61   0  16   0  83   0  53   0 123   0 157
    0  69   0 158   0  61   0  16   0  70   0  16   0  65   0 156   0  43
    0 102   0  46   0  16   0 123   0 156   0  47   0 102   0  56   0  46
    0 147   0  16   0 138 

## Create payload for Triton client
* gRPC 클라이언트를 위한 입력 데이터를 준비하는 함수 정의

In [12]:
import numpy as np
import tritonclient.grpc as grpcclient

#1.함수 정의 및 배치 크기 확인
def create_client_payload(x, x_length, noise_scale, length_scale, noise_scale_w):
    inputs = []
    batch_size = x.shape[0]     # Determine the batch size from x
    
#2.텍스트 입력 설정:
    # x input
    x_input = grpcclient.InferInput('x', x.shape, "INT64")
    x_input.set_data_from_numpy(x)
    inputs.append(x_input)

#3.텍스트 길이 입력 설정:
    # x_length input
    x_length_input = grpcclient.InferInput('x_length', [batch_size, 1], "INT64")
    x_length_input.set_data_from_numpy(x_length.reshape(batch_size, 1))
    inputs.append(x_length_input)

#4.생성 파라미터 입력 설정:
    # 음성 변화도
    noise_scale_input = grpcclient.InferInput('noise_scale', [batch_size, 1], "FP32")
    noise_scale_input.set_data_from_numpy(np.full((batch_size, 1), noise_scale, dtype=np.float32))
    inputs.append(noise_scale_input)

    # 발화 속도
    length_scale_input = grpcclient.InferInput('length_scale', [batch_size, 1], "FP32")
    length_scale_input.set_data_from_numpy(np.full((batch_size, 1), length_scale, dtype=np.float32))
    inputs.append(length_scale_input)

    # 음성 다양성
    noise_scale_w_input = grpcclient.InferInput('noise_scale_w', [batch_size, 1], "FP32")
    noise_scale_w_input.set_data_from_numpy(np.full((batch_size, 1), noise_scale_w, dtype=np.float32))
    inputs.append(noise_scale_w_input)
    
#5. 형태 출력:
    print("x data shape:", x.shape)
    print("x_length shape:", x_length.reshape(batch_size, 1).shape)
    print("noise_scale shape:", (batch_size, 1))
    print("length_scale shape:", (batch_size, 1))
    print("noise_scale_w shape:", (batch_size, 1))
    
    return inputs

#6. 함수 사용 예시:
inputs = create_client_payload(x=x_np, 
                      x_length=x_length_np, 
                      noise_scale=noise_scale_data, 
                      length_scale=length_scale_data,
                      noise_scale_w=noise_scale_w_data)
    


x data shape: (1, 2048)
x_length shape: (1, 1)
noise_scale shape: (1, 1)
length_scale shape: (1, 1)
noise_scale_w shape: (1, 1)


##  Create output variable for Ttiton client

In [13]:
outputs = []
outputs.append(grpcclient.InferRequestedOutput('OUTPUT_0'))


# 6. inference on Triton Docker
* TRITON클라이언트를 사용해 실제 추론을 수행하는 함수를 호출하는 부분


In [14]:
# model_name 변수 확인
print(model_name)  # "tts-vits"가 맞는지 확인

# 사용 가능한 모델 목록 확인
model_list = triton_client.get_model_repository_index()
print("Available models:", model_list)

tts-vits
Available models: models {
  name: "tts-vits"
  version: "1"
  state: "READY"
}



In [15]:
# 서버가 실행 중인지 확인
if triton_client.is_server_ready():
    print("Server is ready")
else:
    print("Server is not ready")

# 모델이 로드되었는지 확인
if triton_client.is_model_ready("tts-vits"):
    print("Model is ready")
else:
    print("Model is not ready")

Server is ready
Model is ready


In [16]:
from local_utils.triton_util import infer_triton_client

result = infer_triton_client(triton_client, model_name, inputs, outputs)

In [17]:
output0_data = result.as_numpy('OUTPUT_0')
output0_data.shape

(1, 1, 798464)

In [21]:
# output0_data[0,0][-500:-1]

In [22]:
audio = output0_data[0,0]
audio

ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=False))

In [26]:
def trim_audio_output(audio_output, x_np, hps):
    """
    패딩이 제거된 실제 오디오 부분만 추출
    
    Parameters:
    - audio_output: 생성된 오디오 numpy array (1, 1, N)
    - x_np: 입력 텍스트 배열 (패딩 포함)
    - hps: 하이퍼파라미터
    """
    # 실제 텍스트 길이 계산 (마지막 non-zero 위치)
    last_token_pos = np.nonzero(x_np[0])[0][-1]
    real_text_length = last_token_pos + 1
    max_text_length = x_np.shape[1]  # 2048
    
    # 오디오 길이
    total_audio_length = audio_output.shape[2]
    
    # 실제 텍스트 비율만큼의 오디오 길이 계산
    text_ratio = real_text_length / max_text_length
    expected_audio_length = int(total_audio_length * text_ratio)
    
    # 결과 출력
    print(f"텍스트 길이 (패딩 제외): {real_text_length}")
    print(f"텍스트 최대 길이: {max_text_length}")
    print(f"텍스트 비율: {text_ratio:.3f}")
    print(f"전체 오디오 길이: {total_audio_length}")
    print(f"잘라낼 오디오 길이: {expected_audio_length}")
    
    # 오디오 트리밍 (shape 유지)
    trimmed_audio = audio_output[:, :, :expected_audio_length]
    
    return trimmed_audio

In [27]:
# audio_output shape: (1, 1, 798464)
trimmed_audio = trim_audio_output(output0_data, x_np, hps)
print("Trimmed audio shape:", trimmed_audio.shape)

# 필요한 경우 shape 변경
final_audio = trimmed_audio[0, 0]  # shape: (N,)

텍스트 길이 (패딩 제외): 964
텍스트 최대 길이: 2048
텍스트 비율: 0.471
전체 오디오 길이: 798464
잘라낼 오디오 길이: 375839
Trimmed audio shape: (1, 1, 375839)


In [29]:
ipd.display(ipd.Audio(final_audio, rate=hps.data.sampling_rate, normalize=False))